In [ ]:
import os
files = [file for file in os.listdir("data/pgn") if file.endswith(".pgn")]

In [ ]:
import os
os.makedirs("data/pgn", exist_ok=True)
!pip install tqdm
!pip install python-chess
!pip install chess
!pip install torch torchvision


In [ ]:
len(files)

6

In [ ]:
from chess import pgn
def load_pgn(file_path):
  games = []
  with open(file_path, "r") as pgn_file:
    while True:
      game = pgn.read_game(pgn_file)
      if game is None:
        break
      games.append(game)
  return games

In [ ]:
from tqdm import tqdm
LIMIT_OF_FILES = min(len(files),24)
games = []
i = 1
for file in tqdm(files):
  games.extend(load_pgn(f"data/pgn/{file}"))
  if(i>= LIMIT_OF_FILES):
    break
  i += 1

 83%|████████▎ | 5/6 [00:00<00:00,  9.08it/s]


In [ ]:
len(games)

187

In [ ]:
import numpy as np
from chess import Board
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
def board_to_matrix(board:Board):
  matrix = np.zeros((8,8,12))
  piece_map = board.piece_map()
  for square ,piece in piece_map.items():
    row,col = divmod(square,8)
    piece_type = piece.piece_type -1
    piece_color = 0 if piece.color else 6
    matrix[row,col,piece_type+piece_color] = 1
  return matrix
def create_input_for_nn(games):
  x = []
  y = []
  for game in games:
    board = game.board()
    for move in game.mainline_moves():
      x.append(board_to_matrix(board))
      y.append(move.uci())
      board.push(move)
  return x,y
def encode_moves(moves):
  move_to_int = {move : idx for idx, move in enumerate(set(moves))}
  return[move_to_int[move]for move in moves], move_to_int

In [ ]:
x ,y  = create_input_for_nn(games)
y, move_to_int = encode_moves(y)
y = to_categorical(y,num_classes=len(move_to_int))
x = np.array(x)


In [ ]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])
model.compile(optimizer = Adam(), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()
model.fit(x,y,epochs = 50 , validation_split = 0.1,batch_size = 64)
model.save("model/chess_model.keras")


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 6, 6, 64)       │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1583)           │       406,831 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,012,207 (3.86 MB)

 Trainable params: 1,012,207 (3.86 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
214/214 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.0139 - loss: 6.8083 - val_accuracy: 0.0184 - val_loss: 6.3497
Epoch 2/50
214/214 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - accuracy: 0.0252 - loss: 6.1989 - val_accuracy: 0.0435 - val_loss: 6.1699
Epoch 3/50
214/214 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.0575 - loss: 5.8081 - val_accuracy: 0.0652 - val_loss: 6.1137
Epoch 4/50
214/214 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - accuracy: 0.0815 - loss: 5.3221 - val_accuracy: 0.0804 - val_loss: 6.3033
Epoch 5/50
214/214 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.1158 - loss: 4.7779 - val_accuracy: 0.0817 - val_loss: 6.6713
Epoch 6/50
214/214 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.1598 - loss: 4.0950 - val_accuracy: 0.0830 - val_loss: 7.1457
Epoch 7/50
214/214 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.2057 - loss: 3.5005 - val_accuracy: 0.0896 - val_loss: 7.7164
Epoch 8/50
214/214 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.2778 - loss: 2.9580 - val

In [ ]:
import os
os.makedirs("model", exist_ok=True)

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("model/chess_model.keras")

In [ ]:
int_to_move = dict(zip(move_to_int.values(), move_to_int.keys()))

def predict_next_moves(board:Board):
  board_matrix = board_to_matrix(board).reshape(1,8,8,12)
  predictions = model.predict(board_matrix)
  legal_moves = list(board.legal_moves)
  legal_moves_uci = [move.uci() for move in legal_moves]
  sorted_indices = np.argsort(predictions)[::-1]
  for move_index in sorted_indices:
    move = int_to_move[move_index]
    if move in legal_moves_uci:
      return move
  return None

In [ ]:
board = Board()

In [ ]:
print("Board before")
print(board)
next_move = predict_next_moves(board)
board.push_uci(next_move)
print("Board after", next_move)
print("board after prediction")
print(board)


Board before
r . b . k b . r
. p q . . p p p
p . p . p n . .
. . . . . . . .
. . . . P P . .
. . N B . . . .
P P P . . . P P
R . B Q K . . R
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Board after e4e5
board after prediction
r . b . k b . r
. p q . . p p p
p . p . p n . .
. . . . P . . .
. . . . . P . .
. . N B . . . .
P P P . . . P P
R . B Q K . . R


In [ ]:
print(str(pgn.Game.from_board(board)))

[Event "?"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "?"]
[Black "?"]
[Result "*"]

1. e4 c5 2. Nf3 e6 3. Nc3 Qc7 4. d4 cxd4 5. Nxd4 a6 6. Bd3 Nf6 7. f4 Nc6 8. Nxc6 dxc6 9. e5 *
